<a href="https://colab.research.google.com/github/newton143/Econ8310/blob/master/Assignment3_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import plotly.express as px

In [2]:
# Data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load training and test data
train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

100%|██████████| 26.4M/26.4M [00:01<00:00, 18.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 301kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.58MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.0MB/s]


In [6]:
# FashionNet Neural Network
class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.flatten = nn.Flatten()
        self.model = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.model(x)


In [8]:
# Initialize model, loss, and optimizer
model = FashionNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [16]:
# Training Loop
def train(model, train_loader, loss_fn, optimizer, epochs=50):
    for epoch in range(epochs):
        for batch, (X, y) in enumerate(train_loader):
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                print(f"Epoch {epoch+1}, Batch {batch}, Loss: {loss.item():.4f}")
    print("Training complete!")


In [17]:
# Evaluate the model
def test(model, test_loader, loss_fn):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in test_loader:
            pred = model(X).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [18]:
# Train and test
train(model, train_loader, loss_fn, optimizer, epochs=50)
test(model, test_loader, loss_fn)

Epoch 1, Batch 0, Loss: 0.1950
Epoch 1, Batch 100, Loss: 0.4569
Epoch 1, Batch 200, Loss: 0.2775
Epoch 1, Batch 300, Loss: 0.1993
Epoch 1, Batch 400, Loss: 0.3050
Epoch 1, Batch 500, Loss: 0.2240
Epoch 1, Batch 600, Loss: 0.2345
Epoch 1, Batch 700, Loss: 0.1802
Epoch 1, Batch 800, Loss: 0.1977
Epoch 1, Batch 900, Loss: 0.3670
Epoch 2, Batch 0, Loss: 0.3000
Epoch 2, Batch 100, Loss: 0.3017
Epoch 2, Batch 200, Loss: 0.1998
Epoch 2, Batch 300, Loss: 0.1921
Epoch 2, Batch 400, Loss: 0.2300
Epoch 2, Batch 500, Loss: 0.2214
Epoch 2, Batch 600, Loss: 0.1973
Epoch 2, Batch 700, Loss: 0.3401
Epoch 2, Batch 800, Loss: 0.1790
Epoch 2, Batch 900, Loss: 0.2023
Epoch 3, Batch 0, Loss: 0.2065
Epoch 3, Batch 100, Loss: 0.4187
Epoch 3, Batch 200, Loss: 0.2891
Epoch 3, Batch 300, Loss: 0.3897
Epoch 3, Batch 400, Loss: 0.3281
Epoch 3, Batch 500, Loss: 0.1484
Epoch 3, Batch 600, Loss: 0.1719
Epoch 3, Batch 700, Loss: 0.1780
Epoch 3, Batch 800, Loss: 0.2630
Epoch 3, Batch 900, Loss: 0.2802
Epoch 4, Batch 0

In [19]:
# Save model weights
torch.save(model.state_dict(), "Assignment3.pth")

In [20]:
# Load and test the model using saved weights
def load_and_test():
    model.load_state_dict(torch.load("Assignment3.pth"))
    model.eval()
    test(model, test_loader, loss_fn)

load_and_test()

Model loaded successfully!
Test Accuracy: 87.72%
